In [79]:
from dotenv import load_dotenv
import os

host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
user_mysql = os.getenv("DB_MYSQL_USERNAME")

In [80]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi


# Provide the connection details
hostname = host
port = 27017  # Default MongoDB port
username = user # If authentication is required
password = password # If authentication is required

# Create a MongoClient instance
client = MongoClient(hostname, port, username=username, password=password,server_api=ServerApi('1'))

                          
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [17]:
db = client["db_produtos"]
collection = db["produtos"]
client.list_database_names()

['admin', 'config', 'db_produtos', 'local']

In [5]:
product = {"produto": "computador", "quantidade": 77}
collection.insert_one(product)

InsertOneResult(ObjectId('65a985247f9a416ead71c9f7'), acknowledged=True)

In [14]:
collection.find_one()

{'_id': ObjectId('65a986c87f9a416ead71c9f8'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'lat': -13.29,
 'lon': -41.71}

In [7]:
client.list_database_names()

['admin', 'config', 'db_produtos', 'local']

In [8]:
import requests

response = requests.get("https://labdados.com/produtos")
response.json()
len(response.json())

9435

In [9]:
docs = collection.insert_many(response.json())

In [10]:
len(docs.inserted_ids)

9435

In [12]:
id_remove = collection.find_one()["_id"]

In [13]:
collection.delete_one({"_id":id_remove})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [15]:
client.close()

In [20]:
collection.find_one()

{'_id': ObjectId('65a986c87f9a416ead71c9f8'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'Latitude': -13.29,
 'Longitude': -41.71}

In [19]:
collection.update_many({}, {"$rename":{"lat":"Latitude","lon":"Longitude"}})

UpdateResult({'n': 9435, 'nModified': 9435, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [22]:
collection.distinct("Categoria do Produto")

['brinquedos',
 'eletrodomesticos',
 'eletronicos',
 'esporte e lazer',
 'instrumentos musicais',
 'livros',
 'moveis',
 'utilidades domesticas']

In [23]:
query = {"Categoria do Produto" : "livros"}
lista_livros = []
for doc in collection.find(query):
    lista_livros.append(doc)


In [25]:
import pandas as pd

In [26]:
df_livros = pd.DataFrame(lista_livros)

In [27]:
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65a986c87f9a416ead71c9f8,Modelagem preditiva,livros,92.45,5.609697,01/01/2020,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,65a986c87f9a416ead71c9f9,Iniciando em programação,livros,43.84,0.000000,01/01/2020,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,65a986c87f9a416ead71ca0b,Iniciando em programação,livros,63.25,3.894137,01/01/2022,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,65a986c87f9a416ead71ca19,Ciência de dados com python,livros,86.13,5.273176,01/02/2021,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,65a986c87f9a416ead71ca1b,Ciência de dados com python,livros,72.75,1.458158,01/02/2021,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [33]:
df_livros["Data da Compra"] = pd.to_datetime(df_livros["Data da Compra"], format="%d/%m/%Y")
df_livros.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _id                     742 non-null    object        
 1   Produto                 742 non-null    object        
 2   Categoria do Produto    742 non-null    object        
 3   Preço                   742 non-null    float64       
 4   Frete                   742 non-null    float64       
 5   Data da Compra          742 non-null    datetime64[ns]
 6   Vendedor                742 non-null    object        
 7   Local da compra         742 non-null    object        
 8   Avaliação da compra     742 non-null    int64         
 9   Tipo de pagamento       742 non-null    object        
 10  Quantidade de parcelas  742 non-null    int64         
 11  Latitude                742 non-null    float64       
 12  Longitude               742 non-null    float64   

In [34]:

df_livros["Data da Compra"] = df_livros["Data da Compra"].dt.strftime("%Y-%m-%d")
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65a986c87f9a416ead71c9f8,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,65a986c87f9a416ead71c9f9,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,65a986c87f9a416ead71ca0b,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,65a986c87f9a416ead71ca19,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,65a986c87f9a416ead71ca1b,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [35]:
df_livros.to_csv('./data/tabela_livros.csv', index=False)

In [36]:
query = {"Data da Compra" : {"$regex" : "/202[1-9]"}}
lista_produtos = []
for produto in collection.find(query):
    lista_produtos.append(produto)

In [37]:
df_produtos = pd.DataFrame(lista_produtos)

df_produtos["Data da Compra"] = pd.to_datetime(df_produtos["Data da Compra"], format="%d/%m/%Y")
df_produtos.info()

df_produtos["Data da Compra"] = df_produtos["Data da Compra"].dt.strftime("%Y-%m-%d")
df_produtos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _id                     6574 non-null   object        
 1   Produto                 6574 non-null   object        
 2   Categoria do Produto    6574 non-null   object        
 3   Preço                   6574 non-null   float64       
 4   Frete                   6574 non-null   float64       
 5   Data da Compra          6574 non-null   datetime64[ns]
 6   Vendedor                6574 non-null   object        
 7   Local da compra         6574 non-null   object        
 8   Avaliação da compra     6574 non-null   int64         
 9   Tipo de pagamento       6574 non-null   object        
 10  Quantidade de parcelas  6574 non-null   int64         
 11  Latitude                6574 non-null   float64       
 12  Longitude               6574 non-null   float64 

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65a986c87f9a416ead71ca01,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,65a986c87f9a416ead71ca02,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,65a986c87f9a416ead71ca03,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,65a986c87f9a416ead71ca04,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,65a986c87f9a416ead71ca05,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [38]:
df_produtos.to_csv('./data/tabela_produtos.csv', index=False)

In [39]:
import mysql.connector

In [71]:
cnx = mysql.connector.connect(
    host = host,
    user = user_mysql,
    password=password
)

print(cnx)

In [72]:
cursor = cnx.cursor()


In [70]:
#cursor.execute("CREATE DATABASE  IF NOT EXISTS db_produtos; ")
cursor.close()

True

In [81]:
cursor.execute("SHOW DATABASES;")
for db in cursor:
    print(db)

('db_produtos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


In [52]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS db_produtos.livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               PRIMARY KEY (id));
    )


""")

In [65]:
cursor.execute("USE db_produtos;")

In [66]:
cursor.execute("SHOW TABLES;")
for tb in cursor:
    print(tb)

('livros',)


In [67]:
df_liv = pd.read_csv("data/tabela_livros.csv")
df_liv.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,Latitude,Longitude
0,65a986c87f9a416ead71c9f8,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,65a986c87f9a416ead71c9f9,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,65a986c87f9a416ead71ca0b,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,65a986c87f9a416ead71ca19,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,65a986c87f9a416ead71ca1b,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [68]:
lista_dados = [tuple(row) for i, row in df_liv.iterrows()]

In [74]:
sql  = "INSERT INTO db_produtos.livros VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados)
cnx.commit()

In [75]:
print(cursor.rowcount,"Dados inseridos")

742 Dados inseridos
